In [1]:
import numpy as np
import os
from glob import glob
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.models import Sequential


In [2]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

IMAGE_SIZE = [224, 224]

In [3]:
# Load VGG16 model
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False

In [5]:
dataset_path = 'C:\Coding\skin_disease\dataset-1'

In [6]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [7]:
# Freeze the layers of the VGG16 base model
for layer in vgg.layers:
    layer.trainable = False
 

In [8]:
num_classes = 26  # Replace with your actual number of classes

# Create the model
x = Flatten()(vgg.output)
prediction = Dense(num_classes, activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:

# Model summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [10]:
# Define the batch size
BATCH_SIZE = 32

# Define the validation split
VALIDATION_SPLIT = 0.2
TEST_SPLIT=0.1

# Load the dataset and automatically split into training and validation sets
train_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=123,  # Random seed for validation splitting
    image_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
    batch_size=BATCH_SIZE)

val_ds = image_dataset_from_directory(
    dataset_path,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=123,  # Random seed for validation splitting
    image_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
    batch_size=BATCH_SIZE)
test_ds=image_dataset_from_directory(
    dataset_path,
    validation_split=TEST_SPLIT,
    subset="validation",    
    seed=123,  # Random seed for validation splitting
    image_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
    batch_size=BATCH_SIZE)


Found 4092 files belonging to 26 classes.
Using 3274 files for training.
Found 4092 files belonging to 26 classes.
Using 818 files for validation.
Found 4092 files belonging to 26 classes.
Using 409 files for validation.


In [11]:
test_ds = image_dataset_from_directory(
    dataset_path,
    seed=123,
    image_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]),
    batch_size=BATCH_SIZE)

Found 4092 files belonging to 26 classes.


In [12]:
# Compile the model
model.compile(
    optimizer='adam',  # Or another optimizer of your choice
    loss='sparse_categorical_crossentropy',  # Use 'categorical_crossentropy' if your labels are one-hot encoded
    metrics=['accuracy']  # It's good practice to monitor accuracy during training
)




In [13]:
# Now, you can fit your model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    verbose=1
)

Epoch 1/5




  9/103 [=>............................] - ETA: 3:33 - loss: 33.7410 - accuracy: 0.0938

KeyboardInterrupt: 

In [27]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_ds)
print(f"Test accuracy: {test_accuracy * 100:.2f}%")

RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.